# Working with multiple files

On occasion, we will need to combine more than 2 files using some combination of `UNION` and `JOIN`.  In this lecture, we will show a clean approach to scaling up these operations up to any number of files.  In the process, we will

1. Use `list` comprehensions to process and `UNION` many similar files.
2. Use `dict` comprehensions to store and access many tables by name.

In [27]:
import polars as pl

## Store in `dict` or `list`?

* Natural sequence/order? $\rightarrow$ `list`
    *  Example: Lakes data and years are a natural sequence
* Easier to refer by name? $\rightarrow$ `dict`
    * Baseball files have no order and easier to refer to by name

## Example 1 - Read all baseball database using `dict`

**Task:** Create a `dict` of tables for all tables in the Lahman database

#### Step 1 - Use `glob` to find paths for all CSV files

In [28]:
from glob import glob
files = glob('./data/baseball/*.csv')
files

['./data/baseball\\AllstarFull.csv',
 './data/baseball\\Appearances.csv',
 './data/baseball\\AwardsManagers.csv',
 './data/baseball\\AwardsPlayers.csv',
 './data/baseball\\AwardsShareManagers.csv',
 './data/baseball\\AwardsSharePlayers.csv',
 './data/baseball\\Batting.csv',
 './data/baseball\\BattingPost.csv',
 './data/baseball\\CollegePlaying.csv',
 './data/baseball\\Fielding.csv',
 './data/baseball\\FieldingOF.csv',
 './data/baseball\\FieldingOFsplit.csv',
 './data/baseball\\FieldingPost.csv',
 './data/baseball\\HallOfFame.csv',
 './data/baseball\\HomeGames.csv',
 './data/baseball\\league_ERA.csv',
 './data/baseball\\league_slugging.csv',
 './data/baseball\\Managers.csv',
 './data/baseball\\ManagersHalf.csv',
 './data/baseball\\Parks.csv',
 './data/baseball\\People.csv',
 './data/baseball\\Pitching.csv',
 './data/baseball\\PitchingPost.csv',
 './data/baseball\\Salaries.csv',
 './data/baseball\\Schools.csv',
 './data/baseball\\SeriesPost.csv',
 './data/baseball\\Teams.csv',
 './data/b

#### Step 2 - Make a function to extract the table name

In [48]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball\\([a-zA-Z_]*)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) if FILE_NAME_RE.match(p) else None

[file_name(p) for p in files]

['AllstarFull',
 'Appearances',
 'AwardsManagers',
 'AwardsPlayers',
 'AwardsShareManagers',
 'AwardsSharePlayers',
 'Batting',
 'BattingPost',
 'CollegePlaying',
 'Fielding',
 'FieldingOF',
 'FieldingOFsplit',
 'FieldingPost',
 'HallOfFame',
 'HomeGames',
 'league_ERA',
 'league_slugging',
 'Managers',
 'ManagersHalf',
 'Parks',
 'People',
 'Pitching',
 'PitchingPost',
 'Salaries',
 'Schools',
 'SeriesPost',
 'Teams',
 'TeamsFranchises',
 'TeamsHalf']

#### 4 - Read in the tables.

In [30]:
baseball_db = {file_name(p):pl.read_csv(p, infer_schema_length=10000,ignore_errors=True,truncate_ragged_lines=True) for p in files}

baseball_db

{'AllstarFull': shape: (5_673, 8)
 ┌───────────┬────────┬─────────┬──────────────┬────────┬──────┬─────┬─────────────┐
 │ playerID  ┆ yearID ┆ gameNum ┆ gameID       ┆ teamID ┆ lgID ┆ GP  ┆ startingPos │
 │ ---       ┆ ---    ┆ ---     ┆ ---          ┆ ---    ┆ ---  ┆ --- ┆ ---         │
 │ str       ┆ i64    ┆ i64     ┆ str          ┆ str    ┆ str  ┆ i64 ┆ i64         │
 ╞═══════════╪════════╪═════════╪══════════════╪════════╪══════╪═════╪═════════════╡
 │ kalinal01 ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ DET    ┆ AL   ┆ 1   ┆ 9           │
 │ demaejo01 ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ KC1    ┆ AL   ┆ 0   ┆ null        │
 │ grimbo01  ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ NYA    ┆ AL   ┆ 1   ┆ null        │
 │ howarel01 ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ NYA    ┆ AL   ┆ 0   ┆ null        │
 │ loesbi01  ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ BAL    ┆ AL   ┆ 1   ┆ null        │
 │ …         ┆ …      ┆ …       ┆ …            ┆ …      ┆ …    ┆ …   ┆ …           │
 │ mantijo01 ┆ 2022   ┆ 0      

### We can now access all the tables by name.

In [31]:
# Biggish output
baseball_db

{'AllstarFull': shape: (5_673, 8)
 ┌───────────┬────────┬─────────┬──────────────┬────────┬──────┬─────┬─────────────┐
 │ playerID  ┆ yearID ┆ gameNum ┆ gameID       ┆ teamID ┆ lgID ┆ GP  ┆ startingPos │
 │ ---       ┆ ---    ┆ ---     ┆ ---          ┆ ---    ┆ ---  ┆ --- ┆ ---         │
 │ str       ┆ i64    ┆ i64     ┆ str          ┆ str    ┆ str  ┆ i64 ┆ i64         │
 ╞═══════════╪════════╪═════════╪══════════════╪════════╪══════╪═════╪═════════════╡
 │ kalinal01 ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ DET    ┆ AL   ┆ 1   ┆ 9           │
 │ demaejo01 ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ KC1    ┆ AL   ┆ 0   ┆ null        │
 │ grimbo01  ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ NYA    ┆ AL   ┆ 1   ┆ null        │
 │ howarel01 ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ NYA    ┆ AL   ┆ 0   ┆ null        │
 │ loesbi01  ┆ 1957   ┆ 0       ┆ NLS195707090 ┆ BAL    ┆ AL   ┆ 1   ┆ null        │
 │ …         ┆ …      ┆ …       ┆ …            ┆ …      ┆ …    ┆ …   ┆ …           │
 │ mantijo01 ┆ 2022   ┆ 0      

In [32]:
baseball_db['Teams']

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
i64,str,str,str,str,i64,i64,i64,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,i64,i64,i64,str,str,str
1871,"""NA""","""BS1""","""BNA""",null,3,31,null,20,10,null,null,"""N""",null,401,1372,426,70,37,3,60,19,73,16,null,null,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,"""Boston Red Stockings""","""South End Grounds I""",null,103,98,"""BOS""","""BS1""","""BS1"""
1871,"""NA""","""CH1""","""CNA""",null,2,28,null,19,9,null,null,"""N""",null,302,1196,323,52,21,10,60,22,69,21,null,null,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,"""Chicago White Stockings""","""Union Base-Ball Grounds""",null,104,102,"""CHI""","""CH1""","""CH1"""
1871,"""NA""","""CL1""","""CFC""",null,8,29,null,10,19,null,null,"""N""",null,249,1186,328,35,40,7,26,25,18,8,null,null,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,"""Cleveland Forest Citys""","""National Association Grounds""",null,96,100,"""CLE""","""CL1""","""CL1"""
1871,"""NA""","""FW1""","""KEK""",null,7,19,null,7,12,null,null,"""N""",null,137,746,178,19,8,2,33,9,16,4,null,null,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,"""Fort Wayne Kekiongas""","""Hamilton Field""",null,101,107,"""KEK""","""FW1""","""FW1"""
1871,"""NA""","""NY2""","""NNA""",null,5,33,null,16,17,null,null,"""N""",null,302,1404,403,43,21,1,33,15,46,15,null,null,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.84,"""New York Mutuals""","""Union Grounds (Brooklyn)""",null,90,88,"""NYU""","""NY2""","""NY2"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023,"""NL""","""PIT""","""PIT""","""C""",4,162,81,76,86,"""N""","""N""","""N""","""N""",692,5406,1293,287,31,159,556,1464,117,41,58,43,790,731,4.6,2,2,47,4290,1380,179,596,1363,91,360,0.984,"""Pittsburgh Pirates""","""PNC Park""",1630624,99,101,"""PIT""","""PIT""","""PIT"""
2023,"""NL""","""SDN""","""SDP""","""W""",3,162,81,82,80,"""N""","""N""","""N""","""N""",752,5401,1316,273,14,205,653,1311,137,32,57,44,648,598,3.73,0,0,36,4323,1270,174,557,1445,73,356,0.988,"""San Diego Padres""","""PETCO Park""",3271554,94,93,"""SDP""","""SDN""","""SDN"""
2023,"""NL""","""SFN""","""SFG""","""W""",4,162,81,79,83,"""N""","""N""","""N""","""N""",674,5412,1271,256,13,174,544,1492,57,16,81,39,719,641,4.02,4,2,50,4304,1395,173,403,1359,117,399,0.981,"""San Francisco Giants""","""AT&T Park""",2500153,96,96,"""SFG""","""SFN""","""SFN"""


In [33]:
baseball_db['Batting']

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zwilldu01""",1915,1,"""CHF""","""FL""",150,null,548,65,157,32,7,13,94,24,null,67,65,null,2,18,null,null,null
"""zwilldu01""",1916,1,"""CHN""","""NL""",35,null,53,4,6,1,0,1,8,0,null,4,6,null,0,2,null,null,null
"""zychto01""",2015,1,"""SEA""","""AL""",13,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


## Example 2 - Reading and joining the baseball database using `dict`

**Task:** Collect the number of total hits for each batters in the 2010 season join on their first and last name.

In the second example, we will store the data frames in a `dict`, which will make it easier to join the files by ne

#### Step 1 - Get the files names

* Only need the `Batting.csv` and `People.csv`.  
* Narrow with a RegEx

In [53]:
from glob import glob
files = glob('./data/baseball/*.csv')
files

['./data/baseball\\AllstarFull.csv',
 './data/baseball\\Appearances.csv',
 './data/baseball\\AwardsManagers.csv',
 './data/baseball\\AwardsPlayers.csv',
 './data/baseball\\AwardsShareManagers.csv',
 './data/baseball\\AwardsSharePlayers.csv',
 './data/baseball\\Batting.csv',
 './data/baseball\\BattingPost.csv',
 './data/baseball\\CollegePlaying.csv',
 './data/baseball\\Fielding.csv',
 './data/baseball\\FieldingOF.csv',
 './data/baseball\\FieldingOFsplit.csv',
 './data/baseball\\FieldingPost.csv',
 './data/baseball\\HallOfFame.csv',
 './data/baseball\\HomeGames.csv',
 './data/baseball\\league_ERA.csv',
 './data/baseball\\league_slugging.csv',
 './data/baseball\\Managers.csv',
 './data/baseball\\ManagersHalf.csv',
 './data/baseball\\Parks.csv',
 './data/baseball\\People.csv',
 './data/baseball\\Pitching.csv',
 './data/baseball\\PitchingPost.csv',
 './data/baseball\\Salaries.csv',
 './data/baseball\\Schools.csv',
 './data/baseball\\SeriesPost.csv',
 './data/baseball\\Teams.csv',
 './data/b

#### Step 2 - Make helper functions to get the name from path

In [45]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball/core\\(Batting|People)\.csv$')
is_batting_or_people = lambda p: FILE_NAME_RE.match(p)
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 

[file_name(p) for p in files if is_batting_or_teams(p)]

NameError: name 'is_batting_or_teams' is not defined

In [43]:
dfs = [pl.read_csv(p) for p in files if is_batting_or_people(p)]

dfs

[]

#### Step 3 - Use a comprehension to read in all files

**Note:** The data is small (< 10mb total) so it is safe to read all at once.

In [44]:
dfs = {file_name(p):pl.read_csv(p) for p in files if is_batting_or_teams(p)}
dfs['Batting'].head()

NameError: name 'is_batting_or_teams' is not defined

In [ ]:
dfs['People'].head()

#### Step 4 - Preprocess each file.

In [ ]:
# Filter, select, and aggregate hits for 2010.
hits_in_2010_raw = (dfs['Batting']
                   .select(['yearID', 'playerID', 'H'])
                   .filter(pl.col('yearID') == 2010)
                   .group_by('playerID')
                   .agg(pl.col('H').mean().alias('Total Hits'))
                   )
hits_in_2010_raw.head(2)

In [ ]:
# Grab the first and last names from People.

player_names = (dfs['People']
                .select(['playerID', 'nameFirst', 'nameLast'])
               )
player_names.head(2)

#### Step 4 -- Join the tables

In [45]:
hits_in_2010 = (hits_in_2010_raw 
                .join(player_names, on='playerID', how='left')
                .drop('playerID')
               )
hits_in_2010.head()

Total Hits,nameFirst,nameLast
f64,str,str
0.0,"""Rob""","""Delaney"""
5.0,"""Josh""","""Donaldson"""
79.0,"""Miguel""","""Montero"""
12.0,"""Cory""","""Sullivan"""
1.0,"""Mike""","""Baxter"""


## <font color="red"> Exercise 3.2 </font>

We want to get the total hits allowed for all pitchers during the 2000-2010 seasons.  Use `glob` and a `dict` to collect this information into a table that includes the players first and last names.

In [56]:
# Your code here


import re
FILE_NAME_RE = re.compile(r'^\./data/baseball\\([a-zA-Z_]*)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) if FILE_NAME_RE.match(p) else None

[file_name(p) for p in files]

['AllstarFull',
 'Appearances',
 'AwardsManagers',
 'AwardsPlayers',
 'AwardsShareManagers',
 'AwardsSharePlayers',
 'Batting',
 'BattingPost',
 'CollegePlaying',
 'Fielding',
 'FieldingOF',
 'FieldingOFsplit',
 'FieldingPost',
 'HallOfFame',
 'HomeGames',
 'league_ERA',
 'league_slugging',
 'Managers',
 'ManagersHalf',
 'Parks',
 'People',
 'Pitching',
 'PitchingPost',
 'Salaries',
 'Schools',
 'SeriesPost',
 'Teams',
 'TeamsFranchises',
 'TeamsHalf']

In [61]:
baseball_db = {file_name(p):pl.read_csv(p, infer_schema_length=10000,ignore_errors=True,truncate_ragged_lines=True) for p in files}


In [62]:
baseball_db['People'].head()

ID,playerID,birthYear,birthMonth,birthDay,birthCity,birthCountry,birthState,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,bbrefID,finalGame,retroID
i64,str,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str
1,"""aardsda01""",1981,12,27,"""Denver""","""USA""","""CO""",null,null,null,null,null,null,"""David""","""Aardsma""","""David Allan""",215,75,"""R""","""R""","""2004-04-06""","""aardsda01""","""2015-08-23""","""aardd001"""
2,"""aaronha01""",1934,2,5,"""Mobile""","""USA""","""AL""",2021,1,22,"""USA""","""GA""","""Atlanta""","""Hank""","""Aaron""","""Henry Louis""",180,72,"""R""","""R""","""1954-04-13""","""aaronha01""","""1976-10-03""","""aaroh101"""
3,"""aaronto01""",1939,8,5,"""Mobile""","""USA""","""AL""",1984,8,16,"""USA""","""GA""","""Atlanta""","""Tommie""","""Aaron""","""Tommie Lee""",190,75,"""R""","""R""","""1962-04-10""","""aaronto01""","""1971-09-26""","""aarot101"""
4,"""aasedo01""",1954,9,8,"""Orange""","""USA""","""CA""",null,null,null,null,null,null,"""Don""","""Aase""","""Donald William""",190,75,"""R""","""R""","""1977-07-26""","""aasedo01""","""1990-10-03""","""aased001"""
5,"""abadan01""",1972,8,25,"""Palm Beach""","""USA""","""FL""",null,null,null,null,null,null,"""Andy""","""Abad""","""Fausto Andres""",184,73,"""L""","""L""","""2001-09-10""","""abadan01""","""2006-04-13""","""abada001"""


In [72]:
baseball_db['People'].head()

ID,playerID,birthYear,birthMonth,birthDay,birthCity,birthCountry,birthState,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,bbrefID,finalGame,retroID
i64,str,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str
1,"""aardsda01""",1981,12,27,"""Denver""","""USA""","""CO""",null,null,null,null,null,null,"""David""","""Aardsma""","""David Allan""",215,75,"""R""","""R""","""2004-04-06""","""aardsda01""","""2015-08-23""","""aardd001"""
2,"""aaronha01""",1934,2,5,"""Mobile""","""USA""","""AL""",2021,1,22,"""USA""","""GA""","""Atlanta""","""Hank""","""Aaron""","""Henry Louis""",180,72,"""R""","""R""","""1954-04-13""","""aaronha01""","""1976-10-03""","""aaroh101"""
3,"""aaronto01""",1939,8,5,"""Mobile""","""USA""","""AL""",1984,8,16,"""USA""","""GA""","""Atlanta""","""Tommie""","""Aaron""","""Tommie Lee""",190,75,"""R""","""R""","""1962-04-10""","""aaronto01""","""1971-09-26""","""aarot101"""
4,"""aasedo01""",1954,9,8,"""Orange""","""USA""","""CA""",null,null,null,null,null,null,"""Don""","""Aase""","""Donald William""",190,75,"""R""","""R""","""1977-07-26""","""aasedo01""","""1990-10-03""","""aased001"""
5,"""abadan01""",1972,8,25,"""Palm Beach""","""USA""","""FL""",null,null,null,null,null,null,"""Andy""","""Abad""","""Fausto Andres""",184,73,"""L""","""L""","""2001-09-10""","""abadan01""","""2006-04-13""","""abada001"""


In [77]:
total_hits_allowed = (baseball_db['Pitching']
                      .filter((pl.col('yearID') >= 2000, pl.col('yearID') <= 2010))
                     .select(['playerID','yearID','H'])
                     .group_by(pl.col('playerID','yearID'))
                     .agg(pl.col('H').sum().alias('Total Hits allowed for season')))
playerNames = (baseball_db['People']
              .select(['playerID','nameGiven']))
hits_allowed_named = (total_hits_allowed
                     .join(playerNames, on='playerID', how='left')
                     .select('nameGiven','yearID','Total Hits allowed for season'))
hits_allowed_named.head()

nameGiven,yearID,Total Hits allowed for season
str,i64,i64
"""Abraham""",2004,8
"""Bronson Anthony""",2002,30
"""Kazuhiro""",2002,44
"""Omar Jesus""",2003,134
"""Lance Daniel""",2007,5
